In [1]:
import numpy as np
import itertools
from environment.env import GridEnvDeform
from tqdm import trange

In [2]:
# maze size
N = 2

# thetas deformations (range(a,b),range(c,d))
l0 = 1
h0 = 10
l1 = 1
h1 = 10

maze = np.load(f"maze/maze_{N}.npy")
env = GridEnvDeform(maze,l0,h0,l1,h1)

states = [((x,y,phi),(i,j)) for x in range(1,env.max_shape[0]-1) for y in range(1,env.max_shape[1]-1) for phi in range(4) for i in range(l0,h0) for j in range(l1,h1)] 
positions = [(x,y,phi) for x in range(1,env.max_shape[0]-1) for y in range(1,env.max_shape[1]-1) for phi in range(4)]
actions = [0,1,2,3]
obs = list(itertools.product([0,1], repeat=5))
thetas = [(i,j) for i in range(l0,h0) for j in range(l1,h1)]

state_dict = {state: i for i, state in enumerate(states)}
position_dict = {position: i for i, position in enumerate(positions)}
obs_dict = {obs : i for i, obs in enumerate(obs)}

# Actions are: 0-listen, 1-open-left, 2-open-right
lenS = len(states)
lenP = len(positions)
lenA = len(actions)
lenO = len(obs)

lenS

236196

In [3]:
from stable_baselines3 import DQN
from environment.env import MDPGYMGridEnvDeform

env = MDPGYMGridEnvDeform(maze,l0,h0,l1,h1)
model = DQN("MultiInputPolicy",env)

model.load("agents/pretrained/MDP/DQNsb3_mpqiitkg/model.zip")

/home/flaccagora/.miniconda3/envs/rob/lib/python3.9/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() takes at most 16 arguments (18 given)
  warnings.warn(
/home/flaccagora/.miniconda3/envs/rob/lib/python3.9/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() takes at most 16 arguments (18 given)
  warnings.warn(


In [4]:

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder

def make_env():
    N = 2

    # thetas deformations (range(a,b),range(c,d))
    l0 = 1
    h0 = 10
    l1 = 1
    h1 = 10
    
    maze = np.load(f"maze/maze_{N}.npy")
    env = MDPGYMGridEnvDeform(maze,l0,h0,l1,h1, render_mode="human")

    env = Monitor(env)  # record stats such as returns
    return env

env = DummyVecEnv([make_env])



In [5]:
def eval_agent_mdp(agent,env,num_episodes,max_episode_steps,render):
    """Returns
        - episode_transition: list of list of tuples (s,a,r,s',done), t[i] is the ith episode
        - beliefs: list of beliefs at each time step 
    """
    
    # if render:
    #     env.set_rendering()

    for i in trange(num_episodes):

        s = env.reset()

        totalReward = 0.0
        done = False
        steps = 0
        episode_transitions = []

        while not done and steps < max_episode_steps:

            best_action, _ = agent.predict(s,deterministic=True)

            next_state, reward, done, info = env.step(best_action)

            totalReward += reward            

            if render:
                # print("State", s)
                # print("Action: ", best_action)
                # print("Reward:     " + str(totalReward) + "  ")
                # print("Next State: ", next_state)
                # print("\n")
            
                env.render()

            s = next_state
            steps += 1



In [6]:
eval_agent_mdp(model,env,10,100,True)

 40%|████      | 4/10 [00:10<00:16,  2.70s/it]


error: display Surface quit